# Align and call mutations in PacBio sequencing of viral genes.

This Python Jupyter notebook aligns the PacBio sequencing of the viral genes, calls mutations, and then puts them in numbering based on the viral ORF.

In [ ]:
import os
import warnings

from IPython.display import HTML, display

import alignparse.ccs
import alignparse.consensus
import alignparse.minimap2
import alignparse.targets
import alignparse.utils
from alignparse.constants import CBPALETTE

import dms_variants.plotnine_themes
import dms_variants.utils

import pandas as pd

import plotnine as p9

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
p9.theme_set(dms_variants.plotnine_themes.theme_graygrid())

Import snakemake variables

In [ ]:
amplicons = snakemake.input.amplicons
features = snakemake.input.features
plot_amplicons = snakemake.output.plot_amplicons
plot_alignment = snakemake.output.plot_alignment
ccs_report = snakemake.input.ccs_report
ccs_fastq = snakemake.input.ccs_fastq
runs = snakemake.params.runs
amplicon_to_reference_df = snakemake.input.amplicon_to_reference_df
alignment_stats = snakemake.params.alignment_stats
readstats_csv = snakemake.output.readstats_csv
filter_reasons_csv = snakemake.output.filter_reasons_csv
mutation_df = snakemake.output.mutation_df
threads = snakemake.threads

## Import target amplicons
Here we import target files for amplicons that where made using primers that either aligned to ends of flu sequences ('termini') or primers aligning approximately to the middle of a gene ('mid').

In [ ]:
targetfile = amplicons

nlines_to_show = 100
with open(targetfile) as f:
    print(''.join(next(f) for _ in range(nlines_to_show)))

Feature file defines features in amplicon genbank file.

In [ ]:
feature_parse_specs_file = features
with open(feature_parse_specs_file) as f:
    print(f.read())

In [ ]:
targets = alignparse.targets.Targets(
                seqsfile=targetfile,
                feature_parse_specs=feature_parse_specs_file,
                ignore_feature_parse_specs_keys=['default_2tags_no_parsed_ORF_2',
                                                 'default_2tags',
                                                 'default_8tags'],
                allow_extra_features=True,
                allow_clipped_muts_seqs=True)

Now let's plot all transcript amplicons and save plots as a figure. 

In [ ]:
plot = targets.plot(ax_width=10)

In [ ]:
print(f"Saving figure to {plot_amplicons}")
plot.savefig(plot_amplicons, format="svg")

## Align pacbio reads

Create pacbio run dataframe

In [ ]:
pacbio_runs = pd.DataFrame(
    {'name': runs, 'fastq': ccs_fastq, 'report': ccs_report}
)
display(HTML(pacbio_runs.to_html(index=False)))

Now align all ccs to amplicons and filter for the ones that meet criteria defined in `fluCA09_features.yaml`.

Create an alignparse.minimap2.Mapper to run minimap2 with minimap [options](https://jbloomlab.github.io/alignparse/alignparse.minimap2.html#alignparse.minimap2.OPTIONS_VIRUS_W_DEL) that allow for large deletions in viral sequences. 

In [ ]:
mapper = alignparse.minimap2.Mapper(alignparse.minimap2.OPTIONS_VIRUS_W_DEL)

print(f"Using `minimap2` {mapper.version} with these options:\n"
      + ' '.join(mapper.options))

In [ ]:
align_and_parse_outdir = os.path.join(alignment_stats)

In [ ]:
readstats, aligned, filtered = targets.align_and_parse(
    df=pacbio_runs,
    mapper=mapper,
    outdir=align_and_parse_outdir,
    name_col='name',
    queryfile_col='fastq',
    overwrite=True,  # overwrite any existing output
    to_csv=True,
    ncpus=threads,
)

Save readstats to CSV for future reference.

In [ ]:
readstats.to_csv(readstats_csv)

In [ ]:
readstats = pd.read_csv(readstats_csv)

Here are read alignment statistics telling how many reads have been aligned or filtered out for each reference amplicon.

In [ ]:
readstats.head()

In [ ]:
p = (
    p9.ggplot(readstats.assign(
        category=lambda x: pd.Categorical(x['category'],
                                          x['category'].unique(),
                                          ordered=True),
        is_aligned=lambda x: x['category'].str.contains('aligned'),),
        p9.aes('category', 'count', fill='is_aligned'))
    + p9.geom_bar(stat='identity')
    + p9.facet_wrap('~ name', nrow=1)
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               panel_grid_major_x=p9.element_blank(),
               figure_size=(10, 5),
               )
    + p9.scale_fill_manual(values=CBPALETTE)
)
_ = p.draw()

p9.ggsave(plot=p, filename=plot_alignment, verbose=False)

Group counts by gene across runs. Plot fraction filtered/aligned:

In [ ]:
readstats[['filtered','gene']] = readstats['category'].str.split(' ', expand=True)

In [ ]:
readstats['count_gene'] = readstats.groupby(['filtered','gene'])['count'].transform('sum')
readstats['total_gene'] = readstats.groupby(['gene'])['count'].transform('sum')
readstats['frac'] = readstats['count_gene'] / readstats['total_gene']
readstats.head()

Save fractions to CSV:

In [ ]:
readstats[['gene', 'filtered', 'count', 'total_gene', 'frac']].to_csv('filtered_by_gene.csv')

Plot normalized fractions:

In [ ]:
p = (
    p9.ggplot((readstats
               [['filtered','gene','frac']]
               .drop_duplicates()
               .query('filtered != "unmapped"')),
        p9.aes('gene', 'frac', fill='filtered'))
    + p9.geom_bar(stat='identity')
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               panel_grid_major_x=p9.element_blank(),
               figure_size=(6, 3),
               )
    + p9.scale_fill_manual(values=[CBPALETTE[1],CBPALETTE[0]])
)
_ = p.draw()

Now lets look at the most common reasons why some reads have been filtered out.

In [ ]:
concatenated_ff_df = pd.DataFrame()
for filtered_file in filtered:
    ff = pd.read_csv(filtered[filtered_file], na_filter=False)
    ff['gene'] = filtered_file
    concatenated_ff_df = pd.concat([concatenated_ff_df, ff], axis=0)

Save concatenated_ff_df to CSV for future reference:

In [ ]:
concatenated_ff_df.to_csv(filter_reasons_csv)

In [ ]:
concatenated_ff_df = pd.read_csv(filter_reasons_csv)

In [ ]:
concatenated_ff_df.head()

In [ ]:
p = (
    p9.ggplot(concatenated_ff_df, p9.aes('filter_reason'))
    + p9.geom_bar()
    + p9.facet_wrap('~ gene', ncol=5)
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               figure_size=(18, 8),
               panel_grid_major_x=p9.element_blank(),
               )
    )
_ = p.draw()

Count number of CCS filtered for each gene. Count number of CCS filtered for each reason.

In [ ]:
ff_counts_df = (
    concatenated_ff_df
    .groupby(['gene','filter_reason'])
    .count()
    .reset_index()
    .rename(columns={'name':'count'})
    [['gene','filter_reason','count']])
ff_counts_df['total_gene'] = ff_counts_df.groupby(['gene'])['count'].transform('sum')
ff_counts_df['frac_filtered_reason'] = ff_counts_df['count'] / ff_counts_df['total_gene']
display(ff_counts_df)

Save fractions to CSV:

In [ ]:
ff_counts_df.to_csv('filtered_reasons.csv')

Plot fraction:

In [ ]:
p = (
    p9.ggplot(ff_counts_df, p9.aes(x='filter_reason',
                                   y='frac_filtered_reason',
                                   fill='filter_reason'))
    + p9.geom_bar(stat='identity')
    + p9.facet_wrap('~ gene', ncol=4)
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               figure_size=(18, 10),
               panel_grid_major_x=p9.element_blank(),
               )
    )
_ = p.draw()

In [ ]:
p = (
    p9.ggplot(ff_counts_df
              .query('gene == "PB2mid" or gene == "PB2termini" '
                     'or gene == "PB1mid" or gene == "PB1termini" '
                     'or gene == "PAmid" or gene == "PAtermini"'),
                      p9.aes(x='filter_reason',
                                   y='frac_filtered_reason',
                                   fill='filter_reason'))
    + p9.geom_bar(stat='identity')
    + p9.facet_wrap('~ gene', ncol=2)
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               figure_size=(10, 12),
               panel_grid_major_x=p9.element_blank(),
               )
    )
_ = p.draw()

In [ ]:
p = (
    p9.ggplot(ff_counts_df
              .query('gene == "NPmid" or gene == "NPtermini" '
                     'or gene == "NSmid" or gene == "NStermini"'),
                      p9.aes(x='filter_reason',
                                   y='frac_filtered_reason',
                                   fill='filter_reason'))
    + p9.geom_bar(stat='identity')
    + p9.facet_wrap('~ gene', ncol=2)
    + p9.theme(axis_text_x=p9.element_text(angle=90),
               figure_size=(10, 8),
               panel_grid_major_x=p9.element_blank(),
               )
    )
_ = p.draw()

## Get all mutations in a common ORF numbering system
The above code has called mutations in key features.
Now we need to actually convert these into the ORF-based numbering scheme that we care about.

First, we read a CSV file that maps the numbering of sites in each feature to the ORF numbering:

In [ ]:
amplicon_to_reference = pd.read_csv(amplicon_to_reference_df)
amplicon_to_reference.head()

We extract the names of the features of interest from `amplicon_to_reference`.
The first four columns are `target`, `transcript`, `gene`, `wt_nt`, and `ORF_position` and are not relevant to features, so we will not include them. 

In [ ]:
column_names = amplicon_to_reference.columns.tolist()

assert amplicon_to_reference.columns.tolist()[:5] \
    == ['target', 'transcript', 'gene', 'wt_nt', 'ORF_position'], \
    amplicon_to_reference

parsed_features = column_names[5:]
print(parsed_features)

Here are the aligned files made by `targets.align_and_parse`.

In [ ]:
for aligned_files in aligned:
    print(aligned[aligned_files])

Here we read in each aligned transcript CSV file that was made by `targets.align_and_parse`.
Then we re-number the mutations from the feature-level numbering in that file to an ORF-specific numbering scheme.
Next, we combine all of the mutations from each feature into a single column (`all_mutations_orf_numbered`) that gives all mutations in that CCS, and also add the names of the flu genes and transcripts.
Then we write the resulting data frame of mutations to a CSV.

In [ ]:
orf_numbered_alignments = pd.DataFrame()

for target, alignments_csv in aligned.items():
    
    # Load alignments
    alignments = pd.read_csv(alignments_csv, na_filter=False)
    
    print(f"Analyzing {len(alignments)} aligned CCSs for {target}")
    
    # Intersection of parsed_features and this targets' features
    t = targets.get_target(target)
    target_features = [f.name for f in t.features]
    output_features = [p for p in parsed_features if p in target_features]
    print(f"{target} has the following parsed features:")
    for p in output_features:
        print(p)
    print('\n')
    
    for feature in output_features:
        number_mapping = (
                amplicon_to_reference
                .query('target == @target')
                .query(f"{feature}.notnull()", engine='python')
                .assign(feature_num=lambda x: x[feature].astype(int))
                )
        print(f"Map for {feature}:")
        print(number_mapping)

        renumberer = alignparse.utils.MutationRenumber(
                                      number_mapping=number_mapping,
                                      old_num_col='feature_num',
                                      new_num_col='ORF_position',
                                      wt_nt_col='wt_nt',
                                      err_suffix=f" for {feature} in {target}",
                                      )

        alignments[f"{feature}_mutations_orf_numbered"] = (
                    alignments[f"{feature}_mutations"]
                    .map(renumberer.renumber_muts)
                    )

    alignments['target'] = target
    # aggregate mutations across all features to get all mutations in ORF
    alignments = (
    alignments.assign(all_mutations_orf_numbered=lambda x: x.apply(
                    lambda row: alignparse.utils.sort_mutations([
                            row[f"{feature}_mutations_orf_numbered"]
                            for feature in output_features
                            ]),
                    axis=1,
                    )
            )
    # merge in the viral trandcript and gene names
    .merge(amplicon_to_reference[['target', 'gene',
                                  'transcript']].drop_duplicates(),
           how='left', on='target')
    )
    orf_numbered_alignments = orf_numbered_alignments.append(alignments)



assert orf_numbered_alignments['gene'].notnull().all()

print(f"\nWriting ORF-numbered alignments to {mutation_df}")
orf_numbered_alignments.to_csv(mutation_df, index=False, compression='gzip',)